In [0]:
from pyspark.sql import functions as f
from functools import reduce

In [0]:
# kody, ktere znamenaji relaps
nl_target = [
    #operace
    "51233","51235", "51237", "51239", "51283", "51285", "51287", "51289",
    #radio
    "43415", "43213", "43315", "43633", "43641",
    #centrove
    "L01FD01",
    #chemo
    "L01AA01", "L01BC02", "L01BC06", "L01CA04", "L01CD01", "L01CD02", "L01DB01", "L01DB03", "L01XA02",
    #imuno
    "L04AX03"]

print(nl_target)

In [0]:
rekurze_recidiva_cleansed_nl = spark.read.table("rakathon_hackathon.rekurze.rekurze_initial_set")

In [0]:
rekurze_recidiva_cleansed_nl_death = (rekurze_recidiva_cleansed_nl
        .filter(f.col("vnejsi_pricina_umrti_kod").isNull())
        .withColumn("cause_of_death_BC",f.when(f.col("prvotni_pricina_umrti_kod").rlike("C5[0-9]{2}"), 1).otherwise(0))
        .withColumn("death_after_pl",f.col("time_datum_dg_to_umrti") - f.col("pl_delka"))
        .withColumn("dead_5_years_bc",
                    f.when((f.col("death_after_pl") < 5 * 365) 
                           & (f.col("cause_of_death_BC") == 1),1)
                    .when((f.col("observed_time_window") == 1) 
                          & ((f.col("cause_of_death_BC") == 0)
                          | (f.col("death_after_pl") >= 5 * 365)) , 0)
                    .otherwise(None)
)
        )
display(rekurze_recidiva_cleansed_nl_death)

In [0]:
rekurze_recidiva_cleansed_nl_death_relaps = (rekurze_recidiva_cleansed_nl_death
    .withColumn("pl_nl", (f.col("time_datum_dg_to_zahajeni_nl") - f.col("time_datum_dg_to_zahajeni_pl")))
    .withColumn("new_relaps",
    f.when((
        f.col("nl_kod_lecby_1").isin(nl_target)
        | f.col("nl_kod_lecby_2").isin(nl_target)
        | f.col("nl_kod_lecby_3").isin(nl_target)
        | f.col("nl_kod_lecby_4").isin(nl_target)
        | f.col("nl_kod_lecby_5").isin(nl_target)
        | f.col("nl_kod_lecby_6").isin(nl_target)
        | f.col("nl_kod_lecby_7").isin(nl_target)
        | f.col("nl_kod_lecby_8").isin(nl_target)
        | f.col("nl_kod_lecby_9").isin(nl_target)
        | f.col("nl_kod_lecby_10").isin(nl_target))
        &(f.col("stadium")!= 4),
        1)
    .otherwise(0)
)
    .withColumn("relaps_5_years",f.when((f.col("pl_nl") < 5 * 365) 
                                        & (f.col("new_relaps") == 1),1)
                                .when((f.col("observed_time_window") == 1) 
                                      & ((f.col("new_relaps") == 0)
                                      | (f.col("pl_nl") >= 5 * 365)) , 0)
                                .otherwise(None)
                                )
    
    )
display(rekurze_recidiva_cleansed_nl_death_relaps)

In [0]:
#rekurze_recidiva_cleansed_nl_death_relaps.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("rakathon_hackathon.rekurze.rekurze_death_initial_set_oprava")

In [0]:
display(rekurze_recidiva_cleansed_nl_death)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

# Z prava cenzurovane

In [0]:
rekurze_recidiva_cleansed_nl_death_censured = (rekurze_recidiva_cleansed_nl
        .filter(f.col("vnejsi_pricina_umrti_kod").isNull())
        .withColumn("cause_of_death_BC",f.when(f.col("prvotni_pricina_umrti_kod").rlike("C5[0-9]{2}"), 1).otherwise(0))
        .withColumn("death_after_pl",f.col("time_datum_dg_to_umrti") - f.col("pl_delka"))
        .withColumn("dead_5_years_bc",
                    f.when((f.col("death_after_pl") < 5 * 365) 
                           & (f.col("cause_of_death_BC") == 1)
                           & (f.col("observed_time_window") == 1),1)
                    .when((f.col("observed_time_window") == 1) 
                          & ((f.col("cause_of_death_BC") == 0)
                          | (f.col("death_after_pl") >= 5 * 365)) , 0)
                    .otherwise(None)
)
        )
display(rekurze_recidiva_cleansed_nl_death_censured)


In [0]:
rekurze_recidiva_cleansed_nl_death_relaps_censured = (rekurze_recidiva_cleansed_nl_death_censured
    .withColumn("pl_nl", (f.col("time_datum_dg_to_zahajeni_nl") - f.col("time_datum_dg_to_zahajeni_pl")))
    .withColumn("new_relaps",
    f.when(
        (f.col("nl_kod_lecby_1").isin(nl_target)
        | f.col("nl_kod_lecby_2").isin(nl_target)
        | f.col("nl_kod_lecby_3").isin(nl_target)
        | f.col("nl_kod_lecby_4").isin(nl_target)
        | f.col("nl_kod_lecby_5").isin(nl_target)
        | f.col("nl_kod_lecby_6").isin(nl_target)
        | f.col("nl_kod_lecby_7").isin(nl_target)
        | f.col("nl_kod_lecby_8").isin(nl_target)
        | f.col("nl_kod_lecby_9").isin(nl_target)
        | f.col("nl_kod_lecby_10").isin(nl_target))
        &(f.col("stadium")!= 4),
        1)
    .otherwise(0)
)
    .withColumn("relaps_5_years",f.when((f.col("pl_nl") < 5 * 365) 
                                        & (f.col("new_relaps") == 1)
                                        & (f.col("observed_time_window") == 1),1)
                                .when((f.col("observed_time_window") == 1) 
                                      & ((f.col("new_relaps") == 0)
                                      | (f.col("pl_nl") >= 5 * 365)) , 0)
                                .otherwise(None)
                                )
    .withColumn("dead_relaps_5_years", 
                f.when((f.col("dead_5_years_bc") == 1)|(f.col("relaps_5_years") == 1),1)
                .when((f.col("dead_5_years_bc") == 0)&(f.col("relaps_5_years") == 0),0)
                .otherwise(None)
                )
    
    )
display(rekurze_recidiva_cleansed_nl_death_relaps_censured)

In [0]:
(rekurze_recidiva_cleansed_nl_death_relaps_censured.filter(f.col("relaps_5_years")==1)).count()

In [0]:
(rekurze_recidiva_cleansed_nl_death_relaps.filter(f.col("relaps_5_years")==1)).count()

In [0]:
#rekurze_recidiva_cleansed_nl_death_relaps_censured.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("rakathon_hackathon.rekurze.rekurze_death_initial_set_censured_oprava")